#### 在这一节中我们将会搭建一个深层神经网络，用于多分类问题。在多分类问题中，通常会用到softmax函数，这个函数可以将网络的输出序列转换为对应的概率。比如对于序列：$[x_1,x_2,...,x_n]$，经过softmax之后，其中的$x_i$，变为了$\frac{e^{x_i}}{\sum_{j=1}^ne^{x_j}}$
#### 这一节中，我们将采用的数据集是MNIST数据集，其中训练集60000张图片，测试集10000张图片，每张图片都是28 x 28的灰度图

In [1]:
import torch
import numpy as np
from torch import nn
from torch.autograd import Variable
from torchvision.datasets import mnist  # 导入PyTorch中内置的mnist数据集

In [2]:
# 使用内置函数下载 mnist 数据集
train_set = mnist.MNIST('./data', train=True, download=True)
test_set = mnist.MNIST('./data', train=False, download=True)

9920512it [00:02, 3852493.97it/s]                             


Extracting ./data\MNIST\raw\train-images-idx3-ubyte.gz


32768it [00:00, 35283.41it/s]                           


Extracting ./data\MNIST\raw\train-labels-idx1-ubyte.gz


1654784it [00:00, 2752822.72it/s]                             


Extracting ./data\MNIST\raw\t10k-images-idx3-ubyte.gz


8192it [00:00, 17896.60it/s]            


Extracting ./data\MNIST\raw\t10k-labels-idx1-ubyte.gz
Processing...
Done!


In [6]:
# train_set中包含了图片以及图片对应的数字(标签)，如：
image, label = train_set[0]

In [7]:
image

In [8]:
label

5

In [12]:
# 这里的读入的图片是 PIL 库中的格式，我们可以非常方便地将其转换为 numpy array
image = np.array(image, dtype=np.float32)
print(image)  # 其中的0表示黑色，255表示白色
print(image.shape)

[[  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
    0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
    0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
    0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
    0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
    0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   3.  18.
   18.  18. 126. 136. 175.  26. 166. 255. 247. 127.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.  30.  36.  94. 154. 170. 253.
  253. 253. 253. 253. 225. 172. 253. 242. 195.  64.   0.  

In [13]:
# 数据标准化，直接采用参考文档代码
def data_tf(x):
    x = np.array(x, dtype='float32') / 255
    x = (x - 0.5) / 0.5   # 标准化，这个技巧之后会讲到
    x = x.reshape((-1,))  # 拉平
    x = torch.from_numpy(x)  # 将numpy数组转换为tensor
    return x
# 重新加载数据集，申明定义的数据变换
train_set = mnist.MNIST('./data', train=True, transform=data_tf, download=True)
test_set = mnist.MNIST('./data', train=False, transform=data_tf, download=True)

In [20]:
image, label = train_set[0]
print(image.shape)
print(image)

torch.Size([784])
tensor([-1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
        -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
        -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
        -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
        -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
        -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
        -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
        -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
        -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
        -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
        -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
        -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
        -1.0000, -1.00

In [21]:
# 使用 pytorch 自带的 DataLoader 定义一个数据迭代器
from torch.utils.data import DataLoader
train_data = DataLoader(train_set, batch_size=64, shuffle=True)
test_data = DataLoader(test_set, batch_size=128, shuffle=False)

In [22]:
image, label = next(iter(train_data))

In [23]:
print(image.shape)
print(label.shape)

torch.Size([64, 784])
torch.Size([64])


In [31]:
# 定义模型
class module_net(nn.Module):
    def __init__(self, input_num, h_1, h_2, h_3, output_num):
        super(module_net, self).__init__()
        self.fc_1 = nn.Linear(input_num, h_1)
        self.F = nn.ReLU()
        self.fc_2 = nn.Linear(h_1, h_2)
        self.fc_3 = nn.Linear(h_2, h_3)
        self.fc_4 = nn.Linear(h_3, output_num)
    # 定义前向传播过程
    def forward(self, x):
        out = self.fc_1(x)
        out = self.F(out)
        out = self.fc_2(out)
        out = self.F(out)
        out = self.fc_3(out)
        out = self.F(out)
        out = self.fc_4(out)
        return out
'''
seq_net = nn.Sequential(
          nn.Linear(784, 400),
          nn.ReLU(),
          nn.Linear(400, 200),
          nn.ReLU(),
          nn.Linear(200, 100),
          nn.ReLU(),
          nn.Linear(100, 10))
'''

'\nseq_net = nn.Sequential(\n          nn.Linear(784, 400),\n          nn.ReLU(),\n          nn.Linear(400, 200),\n          nn.ReLU(),\n          nn.Linear(200, 100),\n          nn.ReLU(),\n          nn.Linear(100, 10))\n'

In [47]:
# 在模型定义中我们没有在输出层后面加softmax层，这是因为在我们要用的多分类损失函数nn.CrossEntropyLoss()中会进行softmax
net = module_net(784, 400, 200, 100, 10)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters(), lr=0.1)

In [48]:
# 训练模型
Epochs = 20
for epoch in range(Epochs):
    loss_sum = 0
    acc_sum = 0
    for image, label in train_data:
        image = Variable(image)
        label = Variable(label)
        # 前向传播
        y_pred = net(image)
        loss = criterion(y_pred, label)
        # 反向传播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        loss_sum += loss.data.item()
        _, out = y_pred.max(1)  # 返回每一行最大值对应的下标，就是图片的预测值
        num_correct = (out == label).sum().item()  # 统计预测正确的数量
        acc = num_correct / image.shape[0]  # 得到这一个batch的平均准确率
        acc_sum += acc
    ave_train_loss = loss_sum / len(train_data)
    ave_train_acc = acc_sum / len(train_data)
    loss_sum = 0
    acc_sum = 0
    # 在测试集上检验效果
    net.eval()  # 将模型改为预测模式，eval（）时，pytorch会自动把BN和DropOut固定住，不会取平均，而是用训练好的值。
    for image, label in test_data:
        image = Variable(image)
        label = Variable(label)
        y_pred = net(image)
        loss = criterion(y_pred, label)
        loss_sum += loss.data.item()
        _, out = y_pred.max(1)
        num_correct = (out == label).sum().item()
        acc = num_correct / image.shape[0]
        acc_sum += acc
    ave_test_loss = loss_sum / len(test_data)
    ave_test_acc = acc_sum / len(test_data)
    print('epoch: {:2d}, train loss: {:.4f}, train acc: {:.4f}, test loss: {:.4f}, test acc: {:.4f}'.format(epoch + 1, ave_train_loss, ave_train_acc, ave_test_loss, ave_test_acc))

epoch:  1, train loss: 0.5030, train acc: 0.8379, test loss: 0.3100, test acc: 0.8978
epoch:  2, train loss: 0.1719, train acc: 0.9469, test loss: 0.2374, test acc: 0.9188
epoch:  3, train loss: 0.1181, train acc: 0.9631, test loss: 0.2425, test acc: 0.9233
epoch:  4, train loss: 0.0911, train acc: 0.9726, test loss: 0.0858, test acc: 0.9740
epoch:  5, train loss: 0.0768, train acc: 0.9761, test loss: 0.1013, test acc: 0.9690
epoch:  6, train loss: 0.0654, train acc: 0.9792, test loss: 0.1758, test acc: 0.9429
epoch:  7, train loss: 0.0539, train acc: 0.9827, test loss: 0.0991, test acc: 0.9684
epoch:  8, train loss: 0.0450, train acc: 0.9854, test loss: 0.1642, test acc: 0.9463
epoch:  9, train loss: 0.0403, train acc: 0.9870, test loss: 0.0821, test acc: 0.9767
epoch: 10, train loss: 0.0390, train acc: 0.9873, test loss: 0.0883, test acc: 0.9725
epoch: 11, train loss: 0.0297, train acc: 0.9900, test loss: 0.0937, test acc: 0.9728
epoch: 12, train loss: 0.0242, train acc: 0.9923, test